In [6]:
from api.rest import QueryEngine

qe = QueryEngine() 
a = qe.get_calculation(band_gap_range=[.1,2.1]) 
c = a.first().calculations 
print(listc.keys().first())
c['mp-691133'].get_soap()

AttributeError: 'dict_keys' object has no attribute 'first'

In [2]:
%cd /home/jason/dev/materialsweb


/home/jason/dev/materialsweb
